In [1]:
import tensorflow as tf
from data.data_utils import Dataset, save_dataset
import os
from model import AgeModel
import tensorflow.keras.losses as losses

# Set following config to resolve GPU errors
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
sess.as_default()
print("Tensorflow version used: {}".format(tf.__version__))

Tensorflow version used: 2.3.0


In [2]:
from tensorflow.python.client import device_lib
def check_if_gpu_used():
    gpu_names = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
    if len(gpu_names) >= 1:
        print("Number of GPUs used by Tensorflow: {}".format(len(gpu_names)))
    else:
        print("Tensorflow operates on CPU now.")
check_if_gpu_used()

Number of GPUs used by Tensorflow: 1


In [3]:
import h5py
import numpy as np
from time import time
data_path = os.path.join(os.getcwd(), 'data')
datafile_path = os.path.join(data_path, "dataset.h5")
#save_dataset(data_path, dataset) # Compressing data in h5 format
start = time()
f = h5py.File(datafile_path, "r")
train_samples = np.array(f["train_samples"])
train_labels = np.array(f["train_labels"])
val_samples = np.array(f["val_samples"])
val_labels = np.array(f["val_labels"])
test_samples = np.array(f["test_samples"])
test_labels = np.array(f["test_labels"])
end = time()
f.close()
print("Monitoring compressed data details")
## Training set
print("Training data shape: {}".format(train_samples.shape))
print("Training labels shape: {}".format(train_labels.shape))
## Validation set
print("Validation data shape: {}".format(val_samples.shape))
print("Validation labels shape: {}".format(val_labels.shape))
## Test set
print("Testing data shape: {}".format(test_samples.shape))
print("Testing labels shape: {}".format(test_labels.shape))
print("Time to construct the dataset from compressed file: {:.3f} seconds".format(end - start))

Monitoring compressed data details
Training data shape: (5400, 91, 91, 1)
Training labels shape: (5400,)
Validation data shape: (2315, 91, 91, 1)
Validation labels shape: (2315,)
Testing data shape: (1159, 91, 91, 1)
Testing labels shape: (1159,)
Time to construct the dataset from compressed file: 0.181 seconds


In [4]:
# Model specification
non_regularized_model = []
non_regularized_model.append({"type": "conv2d", "filters": 32, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "conv2d", "filters": 32, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None,"reg_ratio": None})
non_regularized_model.append({"type": "pool", "pool_type": "max", "pool_size": (2,2), "strides": None, "padding": "valid"})
non_regularized_model.append({"type": "conv2d", "filters": 64, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "conv2d", "filters": 64, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "pool", "pool_type": "max", "pool_size": (2,2), "strides": None, "padding": "valid"})
non_regularized_model.append({"type": "conv2d", "filters": 128, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "conv2d", "filters": 128, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "conv2d", "filters": 128, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "pool", "pool_type": "max", "pool_size": (2,2), "strides": None, "padding": "valid"})
non_regularized_model.append({"type": "conv2d", "filters": 256, "kernel_size": (3,3), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "conv2d", "filters": 512, "kernel_size": (1,1), "strides":(1,1), "padding":"valid", "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "pool", "pool_type": "max", "pool_size": (2,2), "strides": None, "padding": "valid"})
non_regularized_model.append({"type": "flatten"})
non_regularized_model.append({"type": "dense", "units": 256, "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "dense", "units": 128, "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "dense", "units": 64, "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})
non_regularized_model.append({"type": "dense", "units": 1, "activation": "relu", "initializer": "xavier", "regularizer": None, "reg_ratio": None})

In [5]:
def get_candidate_model(template_model, reg_ratio):
    model = template_model.copy()
    for layer in model:
        if layer["type"] in ("conv2d", "dense"):
            layer["regularizer"] = "l2"
            layer["reg_ratio"] = reg_ratio
            if layer["type"] == "conv2d":
                layer["batch_norm"] = True
    return model

In [6]:
import matplotlib.pyplot as plt
def visualize_experiment(history):
    losses = history.history["loss"]
    valid_losses = history.history["val_loss"]
    print("Training loss for experiment: {}".format(losses[-1]))
    print("Minimum validation loss value achieved: {}".format(np.amin(valid_losses)))
    plt.plot(range(1, len(losses) + 1), losses, color="blue", label="Training Loss")
    plt.plot(range(1, len(losses) + 1), valid_losses, color="orange", label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss plot for best model candidate")
    plt.legend(loc="upper right")

In [7]:
def initial_training(layer_details, train_data, val_data, lr):
    exp_model = AgeModel(layer_details, "channels_last")
    exp_model.build_comp_graph((None, 91, 91, 1))
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.MeanAbsoluteError()
    exp_model.compile(loss=loss, optimizer=optim)
    history = exp_model.fit(x=train_data[0], y=train_data[1], epochs=100, verbose=2, validation_data=val_data)
    return history

In [ ]:
init_model = get_candidate_model(non_regularized_model, 1e-5)
init_hist = initial_training(init_model, (train_samples, train_labels), (val_samples, val_labels), 1e-4)

In [ ]:
visualize_experiment(init_hist)

In [8]:
from tqdm import tqdm

In [ ]:
def optimize_lr_reg(step_count, template_model, training_samples, training_labels, valid_samples, valid_labels):
    lr_exp = np.random.uniform(-6, -3, step_count)
    reg_exp = np.random.uniform(-4, 1, step_count)
    random_idx = np.random.permutation(training_samples.shape[0])
    results = []
    # Results stored as (val_loss, train_loss, lr, reg_ratio)
    for exp_idx in tqdm(range(step_count)):
        model_metadata = get_candidate_model(template_model, 10**reg_exp[exp_idx])
        exp_model = AgeModel(model_metadata, "channels_last")
        exp_model.build_comp_graph((None, 91, 91, 1))
        optim = tf.keras.optimizers.Adam(learning_rate=10**lr_exp[exp_idx])
        loss = tf.keras.losses.MeanAbsoluteError()
        exp_model.compile(loss=loss, optimizer=optim)
        history = exp_model.fit(x=training_samples[random_idx[:500]], y=training_labels[random_idx[:500]], epochs=5, verbose=0, validation_data=(valid_samples, valid_labels))
        results.append((history.history["val_loss"][-1], history.history["loss"][-1], lr_exp[exp_idx], reg_exp[exp_idx]))
    results.sort(key=lambda x: x[0])
    return results
opt_results = optimize_lr_reg(100, non_regularized_model, train_samples, train_labels, val_samples, val_labels)

In [ ]:
print("Best results obtained")
for result in opt_results[:10]:
    print("Valid Loss: {:.3f}, Train Loss: {:.3f}, lr: {:.3f}, reg ratio: {:.3f}".format(result[0], result[1], result[2], result[3]))

In [9]:
def lr_reg_smaller_range(step_count, template_model, training_samples, training_labels, valid_samples, valid_labels):
    lr_exp = np.random.uniform(-5, -3, step_count)
    reg_exp = np.random.uniform(-5, -2, step_count)
    random_idx = np.random.permutation(training_samples.shape[0])
    results = []
    # Results stored as (val_loss, train_loss, lr, reg_ratio)
    for exp_idx in tqdm(range(step_count)):
        model_metadata = get_candidate_model(template_model, 10**reg_exp[exp_idx])
        exp_model = AgeModel(model_metadata, "channels_last")
        exp_model.build_comp_graph((None, 91, 91, 1))
        optim = tf.keras.optimizers.Adam(learning_rate=10**lr_exp[exp_idx])
        loss = tf.keras.losses.MeanAbsoluteError()
        exp_model.compile(loss=loss, optimizer=optim)
        history = exp_model.fit(x=training_samples[random_idx[:500]], y=training_labels[random_idx[:500]], epochs=5, verbose=0, validation_data=(valid_samples, valid_labels),batch_size=64)
        results.append((history.history["val_loss"][-1], history.history["loss"][-1], lr_exp[exp_idx], reg_exp[exp_idx]))
    results.sort(key=lambda x: x[0])
    return results

In [10]:
opt_stage_2 = lr_reg_smaller_range(30, non_regularized_model, train_samples, train_labels, val_samples, val_labels)

100%|██████████| 30/30 [06:25<00:00, 12.85s/it]


In [12]:
print("Best results obtained")
for result in opt_stage_2[:10]:
    print("Valid Loss: {:.3f}, Train Loss: {:.3f}, lr: {:.3f}, reg ratio: {:.3f}".format(result[0], result[1], result[2], result[3]))

Best results obtained
Valid Loss: 28.818, Train Loss: 9.773, lr: -3.451, reg ratio: -4.183
Valid Loss: 30.020, Train Loss: 11.807, lr: -3.103, reg ratio: -3.039
Valid Loss: 30.384, Train Loss: 9.068, lr: -3.371, reg ratio: -3.851
Valid Loss: 30.747, Train Loss: 21.833, lr: -4.596, reg ratio: -3.910
Valid Loss: 31.172, Train Loss: 11.418, lr: -3.703, reg ratio: -2.977
Valid Loss: 31.184, Train Loss: 9.696, lr: -3.116, reg ratio: -4.914
Valid Loss: 31.209, Train Loss: 11.587, lr: -3.897, reg ratio: -3.718
Valid Loss: 31.333, Train Loss: 9.506, lr: -3.706, reg ratio: -4.776
Valid Loss: 31.602, Train Loss: 14.228, lr: -4.379, reg ratio: -4.605
Valid Loss: 31.716, Train Loss: 10.493, lr: -3.573, reg ratio: -2.907


In [13]:
def optim_training(layer_details, train_data, val_data, lr):
    exp_model = AgeModel(layer_details, "channels_last")
    exp_model.build_comp_graph((None, 91, 91, 1))
    optim = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.MeanAbsoluteError()
    exp_model.compile(loss=loss, optimizer=optim)
    history = exp_model.fit(x=train_data[0], y=train_data[1], epochs=100, verbose=2, validation_data=val_data, batch_size=64)
    return history

In [16]:
init_model = get_candidate_model(non_regularized_model, 1e-3)
init_hist = optim_training(init_model, (train_samples, train_labels), (val_samples, val_labels), 1e-4)

Epoch 1/100
85/85 - 11s - loss: 16.2642 - val_loss: 30.2645
Epoch 2/100
85/85 - 10s - loss: 11.4933 - val_loss: 28.2368
Epoch 3/100
85/85 - 10s - loss: 10.0091 - val_loss: 22.2598
Epoch 4/100
85/85 - 10s - loss: 8.8613 - val_loss: 16.0691
Epoch 5/100
85/85 - 10s - loss: 8.0203 - val_loss: 13.0296
Epoch 6/100
85/85 - 10s - loss: 7.1568 - val_loss: 11.8878
Epoch 7/100
85/85 - 10s - loss: 6.6746 - val_loss: 10.6753
Epoch 8/100
85/85 - 10s - loss: 6.2291 - val_loss: 11.3045
Epoch 9/100
85/85 - 10s - loss: 6.0226 - val_loss: 10.8263
Epoch 10/100
85/85 - 10s - loss: 5.3314 - val_loss: 10.9142
Epoch 11/100
85/85 - 10s - loss: 5.0928 - val_loss: 10.1271
Epoch 12/100
85/85 - 10s - loss: 5.0217 - val_loss: 9.3355
Epoch 13/100
85/85 - 10s - loss: 4.7570 - val_loss: 9.3776
Epoch 14/100
85/85 - 10s - loss: 4.5304 - val_loss: 9.3146
Epoch 15/100
85/85 - 10s - loss: 4.3140 - val_loss: 9.3017
Epoch 16/100
85/85 - 10s - loss: 4.3013 - val_loss: 9.2369
Epoch 17/100
85/85 - 10s - loss: 4.1197 - val_loss: